In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import csv

In [2]:
data = pd.read_csv("Exercises4inputNIP.csv", sep = ";", encoding = "utf-8")

numberNIP = data["NIP"].tolist()
numberNIP

[5250011266,
 5250008198,
 5250008407,
 6750002236,
 5840203239,
 8960005408,
 5250005834]

In [3]:
driver = webdriver.Chrome("/Users/marinajuszczyk/chromedriver")
driver.get("https://wyszukiwarkaregon.stat.gov.pl/appBIR/index.aspx")

solution = pd.DataFrame(columns = ['Name', 'NIP', 'PKDCode', 'PKDName'])

for indNIP in numberNIP:
    
    # enter NIP number
    fieldNIP = driver.find_element_by_id("txtNip")
    fieldNIP.send_keys(indNIP)
    driver.find_element_by_id("btnSzukaj").click()
    driver.implicitly_wait(5) # wait or page will crash
    
    # unroll general list [1]
    xpathName = '//*[@id="divListaJednostek"]/table/tbody/tr/td[1]/a'
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpathName))) # javascript element
    driver.find_element_by_xpath(xpathName).click()
    driver.implicitly_wait(5) # wait or page will crash
    
    # unroll PKD list [2]
    xpathPKD = '//*[@id="praw_butLinkDzial"]'
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpathPKD))) # javascript element
    driver.find_element_by_xpath(xpathPKD).click()

    # get company name, NIP and PKD code & name
    name = driver.find_element_by_xpath('//*[@id="praw_nazwa"]').text
    codeNIP = driver.find_element_by_xpath('//*[@id="praw_nip"]').text
    codePKD = driver.find_element_by_xpath('//*[@id="praw_dzial"]/table[1]/tbody/tr/td[1]').text
    namePKD = driver.find_element_by_xpath('//*[@id="praw_dzial"]/table[1]/tbody/tr/td[2]').text

    solution = solution.append({'Name': name, 'NIP': codeNIP, 'PKDCode': codePKD, 'PKDName' : namePKD}, ignore_index = True)

    driver.find_element_by_id("txtNip").clear()
    time.sleep(3) # time.sleep(1): 1 second break turned out to be too short for this code

In [4]:
solution

,Name,NIP,PKDCode,PKDName
0,UNIWERSYTET WARSZAWSKI,5250011266,8542Z,SZKOŁY WYŻSZE
1,NARODOWY BANK POLSKI,5250008198,6411Z,DZIAŁALNOŚĆ BANKU CENTRALNEGO
2,SZKOŁA GŁÓWNA HANDLOWA W WARSZAWIE,5250008407,8542Z,SZKOŁY WYŻSZE
3,UNIWERSYTET JAGIELLOŃSKI,6750002236,8542Z,SZKOŁY WYŻSZE
4,UNIWERSYTET GDAŃSKI,5840203239,8542Z,SZKOŁY WYŻSZE
5,UNIWERSYTET WROCŁAWSKI,8960005408,8542Z,SZKOŁY WYŻSZE
6,POLITECHNIKA WARSZAWSKA,5250005834,8542Z,SZKOŁY WYŻSZE


In [9]:
solution.to_csv("WS_4bonus_Juszczyk.csv", sep=';', encoding='utf-8', index = False)